# Exploring eBay Car Sales Data
## Introduction

In this project, we'll work with a dataset of used cars from _eBay Kleinanzeigen_, a [classifieds](https://en.wikipedia.org/wiki/Classified_advertising) section of the German eBay website.

The dataset was originally [scraped](https://en.wikipedia.org/wiki/Web_scraping) and uploaded to Kaggle by user [orgesleka](https://www.kaggle.com/orgesleka).
The original dataset isn't available on Kaggle anymore, but you can find it [here](https://data.world/data-society/used-cars-data).

We've made a few modifications from the original dataset:
- We sampled 50,000 data points from the full dataset, to ensure your code runs quickly in our hosted environment
- We dirtied the dataset a bit to more closely resemble what would be expected from a scraped dataset (the version uploaded to Kaggle was cleaned to be easier to work with)

The data dictionary provided with data is as follows:
- `dateCrawled` - When this ad was first crawled. All field-values are taken from this date.
- `name` - Name of the car.
- `seller` - Whether the seller is private or a dealer.
- `offerType` - The type of listing
- `price` - The price on the ad to sell the car.
- `abtest` - Whether the listing is included in an A/B test.
- `vehicleType` - The vehicle Type.
- `yearOfRegistration` - The year in which the car was first registered.
- `gearbox` - The transmission type.
- `powerPS` - The power of the car in PS.
- `model` - The car model name.
- `odometer` - How many kilometers the car has driven.
- `monthOfRegistration` - The month in which the car was first registered.
- `fuelType` - What type of fuel the car uses.
- `brand` - The brand of the car.
- `notRepairedDamage` - If the car has a damage which is not yet repaired.
- `dateCreated` - The date on which the eBay listing was created.
- `nrOfPictures` - The number of pictures in the ad.
- `postalCode` - The postal code for the location of the vehicle.
- `lastSeenOnline` - When the crawler saw this ad last online.

The aim of this project is to clean the data and analyze the included used car listings.

We'll start by importing the libraries we need and reading the dataset into pandas.

In [116]:
import pandas as pd
import numpy as np

autos = pd.read_csv('autos.csv', encoding='Latin-1')

In [117]:
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Based on our initial observations, we can see that:
- There are missing values for the columns `vehicleType`, `gearbox`, `model`, `fuelType`, and `notRepairedDamage`.
- Most of our columns are strings.
- The column names use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscores.

## Cleaning Column Names

We'll convert the column names from camelcase to snakecase. We'll also reword some of the column names to make them more descriptive.

In [118]:
print('Previous column names: ')
print(autos.columns)

col_dict = {
    'dateCrawled': 'date_crawled', 'offerType': 'offer_type', 
    'vehicleType': 'vehicle_type', 'yearOfRegistration': 'registration_year',
    'powerPS': 'power_PS', 'monthOfRegistration': 'registration_month',
    'fuelType': 'fuel_type', 'notRepairedDamage': 'unrepaired_damage',
    'dateCreated': 'ad_created', 'nrOfPictures': 'num_of_pictures',
    'postalCode': 'postal_code', 'lastSeen': 'last_seen'
}

autos.rename(mapper=col_dict, axis=1, inplace=True)
print('\nNew column names: ')
print(autos.columns)

Previous column names: 
Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

New column names: 
Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_PS', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')


## Initial Exploration and Cleaning

Now let's do some basic data exploration to determine what other cleaning tasks need to be done. Initially we will look for:

- Text columns where all or almost all values are the same. These can often be dropped as they don't have useful information for analysis.
- Examples of numeric data stored as text which can be cleaned and converted.

In [119]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-19 17:36:18,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Based on the descriptive statistics for all columns, we found that `price` and `odometer` columns stored numeric values as text. We'll remove any non-numeric characters and rename those columns.

In [120]:
autos['price'] = autos['price'].str.replace('$', '').str.replace(',', '').astype(float)
autos['odometer'] = autos['odometer'].str.replace('km', '').str.replace(',', '').astype(float)
autos.rename(columns={'price': 'price_usd', 'odometer':'odometer_km'}, inplace=True)
autos.head()

,date_crawled,name,seller,offer_type,price_usd,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000.0,control,bus,2004,manuell,158,andere,150000.0,3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,8500.0,control,limousine,1997,automatik,286,7er,150000.0,6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,8990.0,test,limousine,2009,manuell,102,golf,70000.0,7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,4350.0,control,kleinwagen,2007,automatik,71,fortwo,70000.0,6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,1350.0,test,kombi,2003,manuell,0,focus,150000.0,7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Based on the previous descriptive statistics, we also found that several columns have small number of unique values. We can inspect these columns further to check whether they have mostly only one value.

In [121]:
autos['seller'].value_counts()

privat        49999
gewerblich        1
Name: seller, dtype: int64

In [122]:
autos['offer_type'].value_counts()

Angebot    49999
Gesuch         1
Name: offer_type, dtype: int64

In [123]:
autos['abtest'].value_counts()

test       25756
control    24244
Name: abtest, dtype: int64

In [124]:
autos['vehicle_type'].value_counts()

limousine     12859
kleinwagen    10822
kombi          9127
bus            4093
cabrio         3061
coupe          2537
suv            1986
andere          420
Name: vehicle_type, dtype: int64

In [125]:
autos['gearbox'].value_counts()

manuell      36993
automatik    10327
Name: gearbox, dtype: int64

In [126]:
autos['odometer_km'].value_counts()

150000.0    32424
125000.0     5170
100000.0     2169
90000.0      1757
80000.0      1436
70000.0      1230
60000.0      1164
50000.0      1027
5000.0        967
40000.0       819
30000.0       789
20000.0       784
10000.0       264
Name: odometer_km, dtype: int64

In [127]:
autos['fuel_type'].value_counts()

benzin     30107
diesel     14567
lpg          691
cng           75
hybrid        37
andere        22
elektro       19
Name: fuel_type, dtype: int64

In [128]:
autos['unrepaired_damage'].value_counts()

nein    35232
ja       4939
Name: unrepaired_damage, dtype: int64

We can see that `seller` and `offer_type` columns mostly only have one value and thus are candidates to be dropped.

## Exploring the Odometer and Price Columns

Let's continue exploring the data, specifically looking for data that doesn't look right. We'll start by analyzing the `odometer_km` and `price_usd` columns. Here's the steps we'll take:

- Analyze the columns using minimum and maximum values and look for any values that look unrealistically high or low (outliers) that we might want to remove.
- We'll use:
    - `Series.unique().shape` to see how many unique values
    - `Series.describe()` to view min/max/median/mean etc
    - `Series.value_counts()`, with some variations:
        - chained to `.head()` if there are lots of values.
        - Because `Series.value_counts()` returns a series, we can use `Series.sort_index()` with `ascending= True` or `False` to view the highest and lowest values with their counts (can also chain to `head()` here).
- When removing outliers, we can do `df[(df["col"] >= x ) & (df["col"] <= y )]`, but it's more readable to use `df[df["col"].between(x,y)]`

In [129]:
print('Number of values (odometer_km): ', autos['odometer_km'].shape[0])
print('Number of unique values (odometer_km): ', autos['odometer_km'].unique().shape[0])
print('\nDescriptive statistics (odometer_km):')
print(autos['odometer_km'].describe())
print('\nTop 5 values (odometer_km):')
print(autos['odometer_km'].value_counts().head())

print('\n\nNumber of values (price_usd): ', autos['price_usd'].shape[0])
print('Number of unique values (price_usd): ', autos['price_usd'].unique().shape[0])
print('\nDescriptive statistics (price_usd):')
print(autos['price_usd'].describe())
print('\nTop 5 values (price_usd):')
print(autos['price_usd'].value_counts().head())

Number of values (odometer_km):  50000
Number of unique values (odometer_km):  13

Descriptive statistics (odometer_km):
count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

Top 5 values (odometer_km):
150000.0    32424
125000.0     5170
100000.0     2169
90000.0      1757
80000.0      1436
Name: odometer_km, dtype: int64


Number of values (price_usd):  50000
Number of unique values (price_usd):  2357

Descriptive statistics (price_usd):
count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price_usd, dtype: float64

Top 5 values (price_usd):
0.0       1421
500.0      781
1500.0     734
2500.0     643
1200.0     639
Name: price_usd, dtype: int64


The `odometer_km` column looks clean enough. As for the `price_usd` column, the price of `0` is unrealistic. Meanwhile, the price of `100,000,000` is too high. We'll consider the valid price of 1 to 100,000 and  remove the other rows.

In [130]:
autos = (
    autos[autos['price_usd'].between(1, 100000)]
)

print('\n\nNumber of values (price_usd): ', autos['price_usd'].shape[0])
print('Number of unique values (price_usd): ', autos['price_usd'].unique().shape[0])
print('\nDescriptive statistics (price_usd):')
print(autos['price_usd'].describe())
print('\nTop 5 values (price_usd):')
print(autos['price_usd'].value_counts().head())



Number of values (price_usd):  48526
Number of unique values (price_usd):  2309

Descriptive statistics (price_usd):
count    48526.000000
mean      5755.574146
std       7514.501772
min          1.000000
25%       1200.000000
50%       3000.000000
75%       7450.000000
max      99900.000000
Name: price_usd, dtype: float64

Top 5 values (price_usd):
500.0     781
1500.0    734
2500.0    643
1200.0    639
1000.0    639
Name: price_usd, dtype: int64


## Exploring the Date Columns

The columns `registration_month` and `registration_year` are represented as numeric values, so we can use `Series.describe()` method to understand the distribution without any extra data processing.

In [131]:
autos[['registration_month', 'registration_year']].describe()

,registration_month,registration_year
count,48526.000000,48526.000000
mean,5.782694,2004.754935
std,3.686184,88.678603
min,0.000000,1000.000000
25%,3.000000,1999.000000
50%,6.000000,2004.000000
75%,9.000000,2008.000000
max,12.000000,9999.000000


The columns `date_crawled`, `last_seen`, and `ad_created`, however, are represented as string values. Let's first understand how the values in the three string columns are formatted. These columns all represent full timestamp values, like so:

In [132]:
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


To understand the date range, we can extract just the date values, use `Series.value_counts()` to generate a distribution, and then sort by the index.

In [133]:
print('date_crawled (only date):')
print(autos['date_crawled'].str[:10].value_counts(normalize=True, dropna=False).sort_index())

print('\nad_created (only date):')
print(autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False).sort_index())

print('\nlast_seen (only date):')
print(autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False).sort_index())


date_crawled (only date):
2016-03-05    0.025306
2016-03-06    0.014054
2016-03-07    0.036043
2016-03-08    0.033281
2016-03-09    0.033096
2016-03-10    0.032210
2016-03-11    0.032601
2016-03-12    0.036929
2016-03-13    0.015682
2016-03-14    0.036578
2016-03-15    0.034270
2016-03-16    0.029613
2016-03-17    0.031633
2016-03-18    0.012880
2016-03-19    0.034806
2016-03-20    0.037897
2016-03-21    0.037361
2016-03-22    0.032910
2016-03-23    0.032230
2016-03-24    0.029345
2016-03-25    0.031612
2016-03-26    0.032189
2016-03-27    0.031076
2016-03-28    0.034847
2016-03-29    0.034126
2016-03-30    0.033652
2016-03-31    0.031839
2016-04-01    0.033652
2016-04-02    0.035507
2016-04-03    0.038598
2016-04-04    0.036517
2016-04-05    0.013086
2016-04-06    0.003174
2016-04-07    0.001401
Name: date_crawled, dtype: float64

ad_created (only date):
2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
                .

We can also convert them to datetime datatype before using the `describe()` method.

In [134]:
print('date_crawled description:')
print(pd.to_datetime(autos['date_crawled']).dt.date.describe())
print('\nad_created description:')
print(pd.to_datetime(autos['ad_created']).dt.date.describe())
print('\nlast_seen description:')
print(pd.to_datetime(autos['last_seen']).dt.date.describe())

date_crawled description:
count          48526
unique            34
top       2016-04-03
freq            1873
Name: date_crawled, dtype: object

ad_created description:
count          48526
unique            76
top       2016-04-03
freq            1885
Name: ad_created, dtype: object

last_seen description:
count          48526
unique            34
top       2016-04-06
freq           10765
Name: last_seen, dtype: object


## Dealing with Incorrect Registration Year Data

In the previous step, we can see that `registration_year` column contains  odd value: the maximum value is `9000`, which is many years into the future. Because a car can't be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. As for the minimum value, valid year could be somewhere in the first few decades of th 1900s. Thus, our minimum value of `1910` still makes sense.

Let's count the number of listings with cars that fall outside the 1900 - 2016 interval and remove those from our dataset.

In [135]:
min_year = 1900
max_year = 2016

print('Number of all listings:')
print(autos.shape[0])

print('\nNumber of listings with cars that fall outside the 1900-2016 interval:')
print(
    autos[(autos['registration_year'] < min_year) | 
          (autos['registration_year'] > max_year)]
    .shape[0]
)

autos = autos[autos['registration_year'].between(min_year, max_year)]

print('\nNumber of all listings after removal:')
print(autos.shape[0])

Number of all listings:
48526

Number of listings with cars that fall outside the 1900-2016 interval:
1884

Number of all listings after removal:
46642


Next, let's take a look at the distribution of the remaining values

In [136]:
# autos[['registration_month', 'registration_year']].describe()
autos['registration_year'].value_counts(normalize=True).sort_index()

1910    0.000107
1927    0.000021
1929    0.000021
1931    0.000021
1934    0.000043
          ...   
2012    0.028022
2013    0.017130
2014    0.014172
2015    0.008319
2016    0.026028
Name: registration_year, Length: 78, dtype: float64

## Exploring Price by Brand

When working with data on cars, it's natural to explore variations across different car brands. We can use aggregation to understand the brand column. Here's what the process looks like:
- Identify the unique values we want to aggregate by
- Create an empty dictionary to store our aggregate data
- Loop over the unique values, and for each: 
    - Subset the dataframe by the unique values 
    - Calculate the mean of whichever column we're interested in
    - Assign the val/mean to the dict as k/v.

In [137]:
print('Number of unique brands:')
print(len(autos['brand'].unique()))

print('\nDistribution:')
brand_distribution = autos['brand'].value_counts(normalize=True, sort=True)
print(brand_distribution)


Number of unique brands:
40

Distribution:
volkswagen        0.211440
bmw               0.110072
opel              0.107671
mercedes_benz     0.096480
audi              0.086617
ford              0.069937
renault           0.047189
peugeot           0.029866
fiat              0.025664
seat              0.018288
skoda             0.016423
nissan            0.015287
mazda             0.015201
smart             0.014172
citroen           0.014022
toyota            0.012714
hyundai           0.010034
sonstige_autos    0.009712
volvo             0.009155
mini              0.008769
mitsubishi        0.008233
honda             0.007847
kia               0.007075
alfa_romeo        0.006646
suzuki            0.005939
chevrolet         0.005703
porsche           0.005574
chrysler          0.003516
dacia             0.002637
daihatsu          0.002508
jeep              0.002273
subaru            0.002144
land_rover        0.002101
saab              0.001651
jaguar            0.001565
daewoo      

We have a total of 40 unique brands. Instead of aggregating by all brands, we'll only aggregate by those that have over 5% of total values. So, we'll be using the brands: `volkswagen`, `opel`, `bmw`, `mercedes_benz`, `audi`, and `ford`

In [140]:
agg_brand_list = brand_distribution[brand_distribution > 0.05].index

agg_brand_price = {}

for brand in agg_brand_list:
    agg_brand_price[brand] = autos[autos['brand'] == brand]['price_usd'].mean().astype(int)

print('Average price (USD) by brands:')
print(agg_brand_price)

Average price (USD) by brands:
{'volkswagen': 5402, 'bmw': 8201, 'opel': 2975, 'mercedes_benz': 8529, 'audi': 9295, 'ford': 3710}


We can see that on average:
- Opel and Ford are less expensive
- BMW, Mercedes Benz, and Audi are more expensive
- Volkswagen is in between.

For the top 6 brands, let's use aggregation to understand the average mileage for those cars and if there's any visible link with mean price. 

In [141]:
agg_brand_mileage = {}

for brand in agg_brand_list:
    agg_brand_mileage[brand] = autos[autos['brand'] == brand]['odometer_km'].mean().astype(int)

print('Average mileage (km) by brands:')
print(agg_brand_mileage)

Average mileage (km) by brands:
{'volkswagen': 128707, 'bmw': 132633, 'opel': 129310, 'mercedes_benz': 130851, 'audi': 129188, 'ford': 124288}


While our natural instinct may be to display both aggregated series objects and visually compare them, this has a few limitations:
- it's difficult to compare more than two aggregate series objects if we want to extend to more columns
- we can't compare more than a few rows from each series object
- we can only sort by the index (brand name) of both series objects so we can easily make visual comparisons

Instead, we can combine the data from both series objects into a single dataframe (with a shared index) and display the dataframe directly. To do this, we'll need to learn two pandas methods:
- [pandas series constructor](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.html)
- [pandas dataframe constructor](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html)

In [146]:
brand_price_series = pd.Series(agg_brand_price)
df = pd.DataFrame(brand_price_series, columns=['mean_price'])

brand_mileage_series = pd.Series(agg_brand_mileage)
df['mean_mileage'] = brand_mileage_series

df

,mean_price,mean_mileage
volkswagen,5402,128707
bmw,8201,132633
opel,2975,129310
mercedes_benz,8529,130851
audi,9295,129188
ford,3710,124288


## Conclusion

Based on our table from the previous step, we can see that the average mileage does not have much variation between different brands. The average price, on the other hand, varies depending on the brands.